# Классификация намерений

## Постановка задачи


Используя открытые источники и наборы данных решить задачу классификации намерений пользователя. В качестве входных данных поступают реплики пользователя (на русском языке), которые необходимо отнести к одному из классов.


## Подход к решению

Задача классификации намерений очень обширная. Необходимо более четко сформулировать задачу. Чаще всего это многоклассовая классификация.

Если намерения пользователей очень сильно отличаются, то можно воспользоваться регулярными выражениями. Можно создать большое количество регулярных выражений, которые будут извлекать определенные слова из запроса. После сделать подобие рейтинга, присваивая каждому регулярному выражению вес. При каждом запросе оценивать какие регулярные выражения сработали и считать рейтинг каждого класса. После выбрать класс с наибольшим рейтингом.

Если намерения пользователей похожи, или реплики пользователей очень разнообразны, то лучше использовать подход с применением машинного обучения.

В качестве базового и неплохого решения можно рассмотреть следующее:

1. Сделать векторное представление запроса и каждого из намерений. Есть множество подходов как это можно сделать.

а) Использовать классический мешок слов. Для этого необходимо запрос разбить на слова, убрать специальные символы, привести все к нижнему регистру, сделать лемматизацию слов, убрать частые слова, которые не несут смысловой нагрузки, убрать слова, которых нет в нашем словаре. После составить вектор предложения для запроса. 

б) Использовать предобученную модель (например bert) для составления векторного представления слов.

2. Сравнить по косинусному расстоянию запрос с каждым из намерений. Выбрать намерение с наибольшим значением косинусного расстояния.

Более продвинутый подход включает в себя машинного обучение своей модели. 

Для реализации этого подхода необходим подходящий датасет на русском языке. К сожалению, найти такой мне не удалось. Можно также взять датасет на английском, перевести и проверить разметку или создать его с 0.

После дообучить предобученную модель(bert, roberta и другие) на наших данных с добавлением слоев классификации.



## Базовое решение

Создадим наши намерения ввиде словаря

In [22]:
intent_mapping = {
    "Я интересуюсь покупкой нового автомобиля и хотел бы узнать о моделях, которые у вас есть в наличии, а также о текущих акциях и специальных предложениях для новых покупателей.": "Покупка автомобиля",
    "Я рассматриваю возможность покупки подержанного автомобиля и хотел бы узнать, какие автомобили с пробегом у вас доступны, их состояние и историю обслуживания, а также условия гарантии на такие автомобили.": "Покупка автомобиля",
    "Хотел бы получить полную информацию о моделях автомобилей, которые вы предлагаете, их технических характеристиках, комплектациях и ценах, чтобы сделать осознанный выбор.": "Покупка автомобиля",
    "Мне нужно провести плановое техническое обслуживание моего автомобиля, и я хотел бы узнать, какие услуги вы включаете в стандартное ТО, каковы ваши цены и можно ли записаться онлайн.": "Сервисный центр",
    "Мой автомобиль нуждается в диагностике и ремонте, хотелось бы узнать, какие виды ремонта вы выполняете, каковы сроки и стоимость выполнения работ, а также есть ли у вас гарантии на проведенные работы.": "Сервисный центр",
    "Ищу информацию о стоимости и условиях проведения ремонта и обслуживания автомобиля в вашем сервисном центре, включая замену расходных материалов и запчастей.": "Сервисный центр",
    "Интересуюсь возможностью приобретения автомобиля в кредит или лизинг, хотел бы узнать о ваших кредитных программах, условиях процентных ставок, сроках кредитования и требованиях к заемщику.": "Финансовые услуги",
    "Хотел бы получить подробную информацию о страховании автомобиля, какие виды страховки вы предлагаете, с какими страховыми компаниями сотрудничаете и каковы основные условия и тарифы.": "Финансовые услуги",
    "Ищу информацию о различных финансовых программах, которые вы предлагаете для покупки автомобиля, включая автокредитование, лизинг и страхование, чтобы выбрать оптимальный вариант для себя.": "Финансовые услуги"
}

Импорт необходимых библиотек

In [23]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

Создадим класс BertVectorizer. Он будет отвечать за векторное представление слов.

In [24]:
class BertVectorizer:
    def __init__(self, model_name: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def get_vector(self, input_message: str) -> np.array:
        # Токенизация текста
        inputs = self.tokenizer(input_message, return_tensors='pt', padding=True, truncation=True, add_special_tokens=True, return_attention_mask=True)
        # Получение эмбеддингов
        with torch.no_grad():
            outputs = self.model(**inputs)
        # Берем последний скрытый слой, который содержит в себе эмбединги
        embeddings = outputs.last_hidden_state[:, 0, :]
        # Переводим в numpy
        embeddings = embeddings.numpy()
        embeddings = embeddings.flatten()
        return embeddings

Создадим класс IntentClassifier. Он будет отвечать за классификацию интентов.

In [25]:
class IntentClassifier:
    def __init__(self, intent_mapping: dict, model_name: str):
        self.intent_mapping = intent_mapping
        self.bert_model = BertVectorizer(model_name)
        self.intent_vectors = self.vectorize_faq()  # Инициализация векторных представлений вопросов FAQ

    def vectorize_faq(self) -> dict:
        intent_vectors = {}
        for key in self.intent_mapping.keys():
            intent_vectors[key] = self.bert_model.get_vector(key)
        return intent_vectors

    def classify(self, input_message: str) -> dict:
        # Векторизуем входящий вопрос
        input_vector = self.bert_model.get_vector(input_message)
        # Преобразуем словарь векторов в матрицу
        intent_keys = list(self.intent_vectors.keys())
        matrix_faq_vectors = np.array([self.intent_vectors[key] for key in intent_keys])
        matrix_faq_vectors = matrix_faq_vectors.reshape(matrix_faq_vectors.shape[0], -1)
        # Расчет косинусного сходства
        cosine_similarity = self.calculate_cosine_similarity(input_vector, matrix_faq_vectors)
        # Найти ключ FAQ с наибольшим косинусным сходством
        most_similar_index = np.argmax(cosine_similarity)
        most_similar_key = intent_keys[most_similar_index]
        # Возвращаем значение, соответствующее самому похожему ключу
        return self.intent_mapping[most_similar_key]

    def calculate_cosine_similarity(self, vector: np.array, matrix: np.array) -> np.array:
        # Вычисление скалярного произведения между вектором и каждым вектором в матрице
        dot_product = np.dot(matrix, vector)
        # Вычисление нормы (длины) вектора
        vector_norm = np.linalg.norm(vector)
        # Вычисление норм (длин) каждого вектора в матрице
        matrix_norms = np.linalg.norm(matrix, axis=1)
        # Вычисление косинусного сходства
        cosine_similarity = dot_product / (vector_norm * matrix_norms)
        return cosine_similarity

Создадим экземпляр класса

In [26]:
model_name = 'cointegrated/LaBSE-en-ru'
classifier = IntentClassifier(intent_mapping, model_name)

Создадим список вопросов клиента к компании

In [27]:
queries = [
    "Мне нужно провести техобслуживание моего автомобиля. Какие услуги вы предлагаете и сколько это стоит?",
    "Я ищу информацию о подержанных автомобилях, которые у вас есть в наличии. Какие условия и гарантии?",
    "Какие текущие предложения и скидки у вас есть на новые автомобили?",
    "Могу ли я получить подробности о программах автокредитования и условиях процентных ставок?",
    "Интересует информация о страховании автомобилей. Какие виды страховки доступны?",
    "Хочу узнать, какие модели новых автомобилей у вас есть и их технические характеристики.",
    "Машина требует диагностики и ремонта. Как долго это займет и сколько будет стоить?",
    "Можете рассказать о лизинговых программах для покупки автомобилей?",
    "Какие варианты обмена старого автомобиля на новый вы предлагаете?",
    "Ищу информацию о стоимости замены расходных материалов и запчастей в вашем сервисе."
]

Выполним классификацию

In [28]:
for query in queries:
    classification = classifier.classify(query)
    print(f"Запрос: {query}\nКлассификация: {classification}\n")

Запрос: Мне нужно провести техобслуживание моего автомобиля. Какие услуги вы предлагаете и сколько это стоит?
Классификация: Сервисный центр

Запрос: Я ищу информацию о подержанных автомобилях, которые у вас есть в наличии. Какие условия и гарантии?
Классификация: Покупка автомобиля

Запрос: Какие текущие предложения и скидки у вас есть на новые автомобили?
Классификация: Покупка автомобиля

Запрос: Могу ли я получить подробности о программах автокредитования и условиях процентных ставок?
Классификация: Финансовые услуги

Запрос: Интересует информация о страховании автомобилей. Какие виды страховки доступны?
Классификация: Финансовые услуги

Запрос: Хочу узнать, какие модели новых автомобилей у вас есть и их технические характеристики.
Классификация: Покупка автомобиля

Запрос: Машина требует диагностики и ремонта. Как долго это займет и сколько будет стоить?
Классификация: Сервисный центр

Запрос: Можете рассказать о лизинговых программах для покупки автомобилей?
Классификация: Финанс

## Итоги

Модель достаточно хорошо классифицирует сообщения клиента и работает достаточно быстро. При вычислениях на видеокарте работает достаточно быстро. 

Также можно быстро и легко подстроить программу для разных задач, просто изменив intent_mapping.

Из минусов можно отметить что есть как более быстрые, так и более точные алгоритмы. Вероятнее всего что если обучить модель, то результаты будут лучше.

К тому же подобная классификация хорошо работает только при большом различии между классами и их небольшом колличестве. Если выбрать больше намерений пользователя, а также эти намерения будут как-то пересекаться, то точность предсказаний очень сильно упадет